In [19]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import time
import os
import sys
from collections import defaultdict
from urllib2 import urlopen
from bs4 import BeautifulSoup
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize,sent_tokenize
from textblob import TextBlob
from lxml import html
from nltk.corpus import wordnet
from PyDictionary import PyDictionary
from nltk.corpus import stopwords
from string import punctuation
from heapq import nlargest
import nltk
import requests

In [2]:
class MyModelParser(tweepy.parsers.ModelParser):
    def parse(self, method, payload):
        result = super(MyModelParser, self).parse(method, payload)
        result._payload = json.loads(payload)
        return result

In [3]:
def load_api():
    ''' Function that loads the twitter API after authorizing the user. '''

    consumer_key = 'qiOaRLVPldazXmYmF3IaIQw4L'
    consumer_secret = 'FAzcCMF1UUyuNuSeddAA1nDJYPeXm6OhaCD084k1t3BZ0HleCY'
    access_token = '717220472623071233-oeDgXLyYdqT92Mi06aaAGV7EtSExfKS'
    access_secret = 'tVh6WAqVqJ5Pekb3skPON4OD46dyyBAIGOiWjPkZrtglC'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)


In [4]:
def tweet_search(api, query, max_tweets, max_id, since_id):
    ''' Function that takes in a search string 'query', the maximum
        number of tweets 'max_tweets', and the minimum (i.e., starting)
        tweet id. It returns a list of tweepy.models.Status objects. '''

    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
                                    max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
    return searched_tweets, max_id

In [5]:
tweets_data = []
api = load_api()
search_phrases = ['#edpolicy','#edreform', 
                     '#edreformtribe', '#edgap',
                     '#literacy', '#teacherquality',
                     '#urbaned', 'edadmin',
                     '#edpolitics']
maxTweets = 100

In [6]:
for search_phrase in search_phrases:
    for tweets in tweepy.Cursor(api.search,search_phrase,\
                           lang="en",\
                           since='2017-05-31',until='2017-06-06').items(maxTweets):
        #print tweet.created_at, tweet.text
    #print type(tweets)
        tweets_data.append(tweets)
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [7]:
print len(tweets_data)

448


In [8]:
stanford_ner_dir = '/home/dell/stanford-ner-2015-04-20/'
eng_model_filename= stanford_ner_dir + 'classifiers/english.all.3class.distsim.crf.ser.gz'
my_path_to_jar= stanford_ner_dir + 'stanford-ner.jar'

st = StanfordNERTagger(model_filename=eng_model_filename, path_to_jar=my_path_to_jar)
enc = lambda x: x.encode('latin1', errors='ignore')

json_tweet_data = []
for tweets in tweets_data:
    json_tweet_data.append(tweets._json)

In [9]:
dictionary = PyDictionary()
search_words = dictionary.synonym("policy")

/home/dell/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/dell/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [18]:
url = json_tweet_data[0]['entities']['urls'][0]['expanded_url']
print url

http://www.theedadvocate.org/?p=2241


In [13]:
text_list = []
for tweets in json_tweet_data:
    if tweets.has_key('user'):
        expanded_url = tweets['entities']['urls']
        if len(expanded_url)!=0 and expanded_url[0]['expanded_url']:
            url = expanded_url[0]['expanded_url']
            #print url
            try:
                htmltree = html.parse(urlopen(url))
            except urllib2.HTTPError, e:
                print e.fp.read()
            p_tags = htmltree.xpath('//p')
            p_content = [p.text_content() for p in p_tags]
            p_content = str(p_content)
            for word in search_words:
                if word.lower() in p_content.lower():
                    text_list.append(p_content)
                    break
        else:
            text = enc(tweets['text'])
            text = str(text)
            for word in search_words:
                if word.lower() in text.lower():
                    text_list.append(text)
                    break

<html>
<head><title>403 Forbidden</title></head>
<body bgcolor="white">
<center><h1>403 Forbidden</h1></center>
<hr><center>nginx</center>
</body>
</html>

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Access denied | www.inacol.org used Cloudflare to restrict access</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/cf.errors.css" type="text/css" media="screen,projection" />
<!--[if lt IE 9]><li

In [26]:
print len(text_list)
print text_list[0]

115
["CMRubinWorld's Award winning series #teaching #Elearning #FutureReady #edchat #STEAM #schools #parents #K12 #globaled #RubinWorldEDU #GlobalSearch4Ed #highered", '\n        When you tweet with a location, Twitter stores that location. \n        You can switch location on/off before each Tweet and always have the option to delete your location history.\n        Learn more\n      ', '', '', "Here's the URL for this Tweet. Copy it to easily share with friends.", 'Add this Tweet to your website by copying the code below. Learn more', 'Add this video to your website by copying the code below. Learn more', 'Hmm, there was a problem reaching the server. Try again?', 'By embedding Twitter content in your website or app, you are agreeing to the Twitter Developer Agreement and Developer Policy.', u'This timeline is where you\u2019ll spend most of your time, getting instant updates about what matters to you.', '\n    Hover over the profile pic and click the Following button to unfollow any 

In [20]:
class FrequencySummarizer:
    def __init__(self, min_cut=0.1, max_cut=0.9):
        """
         Initilize the text summarizer.
         Words that have a frequency term lower than min_cut 
         or higer than max_cut will be ignored.
        """
        self._min_cut = min_cut
        self._max_cut = max_cut 
        self._stopwords = set(stopwords.words('english') + list(punctuation))

    def _compute_frequencies(self, word_sent):
        """ 
          Compute the frequency of each of word.
          Input: 
           word_sent, a list of sentences already tokenized.
          Output: 
           freq, a dictionary where freq[w] is the frequency of w.
        """
        freq = defaultdict(int)
        for s in word_sent:
            for word in s:
                if word not in self._stopwords:
                    freq[word] += 1
    # frequencies normalization and fitering
        m = float(max(freq.values()))
        for w in freq.keys():
            freq[w] = freq[w]/m
            if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
                del freq[w]
        return freq

    def summarize(self, text, n):
        """
          Return a list of n sentences 
          which represent the summary of text.
        """
        sents = sent_tokenize(text)
        assert n <= len(sents)
        word_sent = [word_tokenize(s.lower()) for s in sents]
        self._freq = self._compute_frequencies(word_sent)
        ranking = defaultdict(int)
        for i,sent in enumerate(word_sent):
            for w in sent:
                if w in self._freq:
                    ranking[i] += self._freq[w]
        sents_idx = self._rank(ranking, n)    
        return [sents[j] for j in sents_idx]

    def _rank(self, ranking, n):
        """ return the first n sentences with highest ranking """
        return nlargest(n, ranking, key=ranking.get)

In [21]:
def getTextFromURL(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    return text

In [22]:
def summarizeURL(url, total_pars):
    url_text = getTextFromURL(url).replace(u"Â", u"").replace(u"â", u"")

    fs = FrequencySummarizer()
    final_summary = fs.summarize(url_text.replace("\n"," "), total_pars)
    return " ".join(final_summary)


In [29]:
summarised_list = [] 
for text in text_list:
    fs = FrequencySummarizer()
    #print text
    final_summary = fs.summarize(text.replace("\n"," "),5)
    final_summary = " ".join(final_summary)
    summarised_list.append(final_summary)

In [31]:
print len(summarised_list)

115


In [32]:
tweets_dict = defaultdict(list)
for text in summarised_list:
    tokenized_text = word_tokenize(text)
    classified_text = st.tag(tokenized_text)
    for term,tag in classified_text:
        if tag == 'LOCATION':
            if term in tweets_dict.keys():
                tweets_dict[term].append(text)
            else:
                tweets_dict[term]=[]
                tweets_dict[term].append(text)
                

In [16]:
positive = defaultdict(list)
negative = defaultdict(list)
neutral = defaultdict(list)
for keys in tweets_dict.keys():
    for tweets in tweets_dict[keys]:
        # pass tweet into TextBlob
        tweet = TextBlob(tweets)

        # output sentiment polarity
        #print keys
        #print tweet.sentiment.polarity

        # determine if sentiment is positive, negative, or neutral
        if tweet.sentiment.polarity < 0:
            #sentiment = "negative"
            if keys in negative.keys():
                negative[keys].append(tweets)
            else:
                negative[keys]=[]
                negative[keys].append(tweets)
        elif tweet.sentiment.polarity == 0:
            #sentiment = "neutral"
            if keys in neutral.keys():
                neutral[keys].append(tweets)
            else:
                neutral[keys]=[]
                neutral[keys].append(tweets)
        else:
            #sentiment = "positive"
            if keys in positive.keys():
                positive[keys].append(tweets)
            else:
                positive[keys]=[]
                positive[keys].append(tweets)

In [17]:
print "positive"+":"+str(len(positive.keys())) 
print "negative"+":"+str(len(negative.keys()))
print "neutral"+":"+str(len(neutral.keys()))
print positive.keys()

positive:55
negative:0
neutral:0
[u'JFK', u'City', u'United', u'District', u'Fairway', u'Washington', u'Philadelphia', u',', u'Faerie', u'Ohio', u'New', u'White', u'Valley', u'East', u'North', u'USA', u'&', u'Japan', u'Virginia', u'Oregon', u'NY', u'Connecticut', u'Street', u'Hartford', u'York', u'House', u'Orleanian', u'Jordan', u'America', u'Ahmedabad', u'Netherlands', u'Berlin', u'Pennsylvania', u'U.S.', u'DC', u'Silicon', u'California', u'Missouri', u'Glendale', u'Columbia', u'Yorkshire', u'Cambridge', u'Office', u'D.C', u'Amazon', u'of', u'India', u'US', u'States', u'China', u'UK', u'Boston', u'Oval', u'Topeka', u'Baldwin']


In [14]:
#print len(text_list)
print text_list[1]